# 1-Exctract

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import csv
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
import multiprocessing

# URL scraping for pages containing car listing

In [43]:
base_url = 'https://www.auctionexport.com'
lis_urls = ['/en/Inventory/Search_Results?Offset=0&vehicleType=2']

itera = 100
for i in range(itera):
    url = base_url + lis_urls[-1]
    res = requests.get(url)
    s = BeautifulSoup(res.content, 'html.parser')
    links = s.find('table', class_='pages_container').find_all('a', class_='page')
    pag = [link['href'] for link in links]
    next_url = s.find('table', class_='pages_container').find('td', class_='right_side').find('a').get('href')
    if next_url in lis_urls:
        break
    lis_urls.extend(pag)
    lis_urls.append(next_url)

# Store the URLs in a file named 'urls_car.csv'

In [44]:
full_urls = []
for url in lis_urls:
    full_url = base_url + url
    full_urls.append(full_url)

df = pd.DataFrame(full_urls)
df = df.drop_duplicates()
df.to_csv('urls_car.csv',index=False,header=False)

# Exctract URL from each page


In [45]:
def scrap_cars(url):
    data = []
    base_url = 'https://www.auctionexport.com'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    items = soup.find_all('div', class_='sv-title-container')
    for item in items:
        tag = item.find('a')
        href = tag['href']
        data.append(base_url + href)
    return data

In [46]:
data = []
with open('urls_car.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        url = ''.join(row)
        data.extend(scrap_cars(url))
        
df_urls = pd.DataFrame(data)
df_urls = df_urls.drop_duplicates()

df_urls.to_csv('info_car.csv',index=False, header=False)

# Exctract car information for each item 

In [ ]:
def scrap_cars_info(url):
    data = []
    car_data = {}
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    veh = soup.find('div', class_='vi-vehicle-details-title-spacer').find('table')
    labels = veh.find_all('td', class_='label')
    values = veh.find_all('td', class_='value')
    for label, value in zip(labels, values):
        car_data[label.text.strip()] = value.text.strip()

    inf = soup.find('div', class_='vi-saleinfo-container')
    labels1 = inf.find_all('td', class_='label')
    values1 = inf.find_all('td', class_='value')
    for label, value in zip(labels1, values1):
        car_data[label.text.strip()] = value.text.strip()

    data.append(car_data)
    return data

if __name__ == '__main__':
    num_processes = multiprocessing.cpu_count()
    pool = multiprocessing.Pool(num_processes)
    data = []

    with open('info_car.csv', 'r') as f:
        reader = csv.reader(f)
        urls = [''.join(row) for row in reader]

    chunk_size = len(urls) // num_processes
    url_chunks = [urls[i:i+chunk_size] for i in range(0, len(urls), chunk_size)]

    # Flatten the list of URL chunks
    flat_url_chunks = [url for chunk in url_chunks for url in chunk]

    results = pool.map(scrap_cars_info, flat_url_chunks)

    for result in results:
        for sublist in result:
            data.extend(sublist)

In [ ]:
flattened_results = [item for sublist in results for item in sublist]
df = pd.DataFrame(flattened_results)
df.to_csv('cars_data_0.csv', index=Fa)

# 2-  Data Preprocessing and Transformation

In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('cars_data_0.csv')
di = pd.read_csv('dataset_info.csv')
df.head()

,VIN #:,Make:,Model:,Year:,Mileage:,Exterior Color:,Interior Color:,Drivetrain:,Engine:,Transmission:,Sale Doc:,Keys:,Unnamed: 12,Lot #:,Auction:,Run:,Time Left:,Sale End Date:,Location:
0,**********6114412,Chevrolet,Equinox Lt,2019,99399,White,Beige,AWD,1.5L Turbo DOHC 4-cylinder,Automatic,NaN,NaN,NaN,198147711,A1 A,28.0,19 hours left,"May 04, 2024 10:00 AM (EST)",In Transit
1,**********U637747,Hyundai,Tucson Sel,2018,71338.0 Actual,BLUE,NaN,Front-wheel Drive,2.0L 4-Cylinder Petrol,AUTOMATIC,SALVAGE TITLE,YES,NaN,197599354,NaN,NaN,6 days left,"May 10, 2024 01:30 AM (EST)","Colorado, USA"
2,**********8128941,Hummer,H3,2008,195198.0 Actual,BLACK,NaN,All wheel drive,3.7L 5-Cylinder Petrol,AUTOMATIC,CERTIFICATE OF TITLE,YES,NaN,200038809,A1 A,27.0,19 hours left,"May 04, 2024 10:00 AM (EST)","New Jersey, USA"
3,**********0214513,Mazda,Cx-9,2010,256234,Red,Beige,AWD,3.7L DOHC 24-valve V6 engine,Automatic,NaN,NaN,NaN,199224369,NaN,NaN,6 days left,"May 10, 2024 12:00 AM (EST)","Ohio, USA"
4,**********H782018,Land Rover,Range Rover Evoque Pure Plus,2013,149732,SILVER,NaN,AWD,"2L I-4 DI, DOHC, VVT, turbo 240HP",Automatic,NaN,NaN,NaN,200159825,NaN,NaN,6 days left,"May 10, 2024 01:30 AM (EST)",In Transit


In [3]:
df.shape

(6890, 19)

In [4]:
di.shape

(1457, 19)

# Task 2.1 Clean Data

In [5]:
#handling for Mileage using linear regression to pridict null values depend on year feature
from sklearn.linear_model import LinearRegression
def handle_missing_value(df, column):
    df_with_mileage = df.dropna(subset=[column])
    df_with_mileage[column] = df_with_mileage[column]
    df_with_mileage[column] = pd.to_numeric(df_with_mileage[column], errors='coerce')
    df_mis_mileage = df[df[column].isnull()]
    x_train = df_with_mileage[['Year']]
    y_train = df_with_mileage[column]
    model = LinearRegression()
    model.fit(x_train, y_train)
    x_missing = df_mis_mileage[['Year']]
    pred_mileage = model.predict(x_missing)
    df_mis_mileage[column] = pred_mileage
    df = pd.concat([df_with_mileage, df_mis_mileage])
    return df

In [6]:
# Handling missing values - Hot Deck Imputation
def hot_deck_imputation(data):
    imputed_data = data.copy()
    for column in imputed_data.columns:
        missing_indices = imputed_data[column].isnull()
        unique_values = imputed_data.loc[~missing_indices, column].unique()
        imputed_data.loc[missing_indices, column] = np.random.choice(unique_values, size=missing_indices.sum())
    return imputed_data

In [7]:
import numpy as np
def clean(df):
    df = df.rename(columns={'Lot #:': 'Lot_no',
                        'VIN #:': 'Vin_no',
                        'Make:': 'Make',
                        'Model:': 'Model',
                        'Year:': 'Year',
                        'Location:': 'Location',
                        'Mileage:': 'Mileage',
                        'Exterior Color:':'Exterior_Color',
                        'Interior Color:': 'Interior_Color',
                        'Drivetrain:': 'Drivetrain',
                        'Engine:': 'Engine',
                        'Transmission:': 'Transmission',
                        'Sale Doc:': 'Sale_Doc',
                        'Keys:': 'Keys',
                        'Unnamed: 12': 'Unnamed',
                        'Auction:': 'Auction',
                        'Run:': 'Run',
                        'Time Left:': 'Time_Left',
                        'Sale End Date:': 'Sale_End_Date'})
    df.drop(columns=['Interior_Color','Unnamed', 'Auction','Run'], inplace=True)
    df['Mileage']=df['Mileage'].str.extract(r'(\d+\.?\d*)')
    df=handle_missing_value(df,'Mileage')
    df['Mileage'] = df['Mileage'].astype(int)
    df['Exterior_Color'].fillna(df['Exterior_Color'].mode()[0], inplace=True)
    df['Exterior_Color']=df['Exterior_Color'].str.upper()
    df['Transmission']=df['Transmission'].str.upper()
    df['Transmission'].replace(['UNKNOWN','MISSING'], np.nan, inplace=True)
    df=hot_deck_imputation(df)
    return df

In [8]:
df = clean(df)
di = clean(di)

C:\Users\AMR ALMARI\AppData\Local\Temp\ipykernel_17504\3845919126.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_mileage[column] = df_with_mileage[column]
C:\Users\AMR ALMARI\AppData\Local\Temp\ipykernel_17504\3845919126.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_mileage[column] = pd.to_numeric(df_with_mileage[column], errors='coerce')
C:\Users\AMR ALMARI\AppData\Local\Temp\ipykernel_17504\3845919126.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

In [9]:
print(df.shape)
print(di.shape)

(6890, 15)
(1457, 15)


In [10]:
df.head()

,Vin_no,Make,Model,Year,Mileage,Exterior_Color,Drivetrain,Engine,Transmission,Sale_Doc,Keys,Lot_no,Time_Left,Sale_End_Date,Location
0,**********6114412,Chevrolet,Equinox Lt,2019,99399,WHITE,AWD,1.5L Turbo DOHC 4-cylinder,AUTOMATIC,CERTIFICATE OF DESTRUCTION(Florida),Master - 1pcs,198147711,19 hours left,"May 04, 2024 10:00 AM (EST)",In Transit
1,**********U637747,Hyundai,Tucson Sel,2018,71338,BLUE,Front-wheel Drive,2.0L 4-Cylinder Petrol,AUTOMATIC,SALVAGE TITLE,YES,197599354,6 days left,"May 10, 2024 01:30 AM (EST)","Colorado, USA"
2,**********8128941,Hummer,H3,2008,195198,BLACK,All wheel drive,3.7L 5-Cylinder Petrol,AUTOMATIC,CERTIFICATE OF TITLE,YES,200038809,19 hours left,"May 04, 2024 10:00 AM (EST)","New Jersey, USA"
3,**********0214513,Mazda,Cx-9,2010,256234,RED,AWD,3.7L DOHC 24-valve V6 engine,AUTOMATIC,SALVAGE(Tennessee),YES,199224369,6 days left,"May 10, 2024 12:00 AM (EST)","Ohio, USA"
4,**********H782018,Land Rover,Range Rover Evoque Pure Plus,2013,149732,SILVER,AWD,"2L I-4 DI, DOHC, VVT, turbo 240HP",AUTOMATIC,PREVIOUS SALVAGE(Michigan),NO,200159825,6 days left,"May 10, 2024 01:30 AM (EST)",In Transit


In [11]:
di.head()

,Vin_no,Make,Model,Year,Mileage,Exterior_Color,Drivetrain,Engine,Transmission,Sale_Doc,Keys,Lot_no,Time_Left,Sale_End_Date,Location
0,**********6114412,Chevrolet,Equinox Lt,2019,99399,WHITE,AWD,1.5L Turbo DOHC 4-cylinder,AUTOMATIC,SALVAGE(Massachusetts),PRESENT,198147711,19 hours left,"May 04, 2024 10:00 AM (EST)",In Transit
1,**********U637747,Hyundai,Tucson Sel,2018,71338,BLUE,Front-wheel Drive,2.0L 4-Cylinder Petrol,AUTOMATIC,SALVAGE TITLE,YES,197599354,6 days left,"May 10, 2024 01:30 AM (EST)","Colorado, USA"
2,**********8128941,Hummer,H3,2008,195198,BLACK,All wheel drive,3.7L 5-Cylinder Petrol,AUTOMATIC,CERTIFICATE OF TITLE,YES,200038809,19 hours left,"May 04, 2024 10:00 AM (EST)","New Jersey, USA"
3,**********0214513,Mazda,Cx-9,2010,256234,RED,AWD,3.7L DOHC 24-valve V6 engine,AUTOMATIC,SALVAGE(Indiana),PRESENT,199224369,6 days left,"May 10, 2024 12:00 AM (EST)","Ohio, USA"
4,**********H782018,Land Rover,Range Rover Evoque Pure Plus,2013,149732,SILVER,AWD,"2L I-4 DI, DOHC, VVT, turbo 240HP",AUTOMATIC,CLEAR(Massachusetts),MISSING,200159825,6 days left,"May 10, 2024 01:30 AM (EST)",In Transit


# TASK 2.2 TRANSFORMATION

In [12]:
#encode categorical variables
def encoding(data,column):

  # Perform one-hot encoding
  one_hot_encoded = pd.get_dummies(data[f'{column}']).astype(int)

  # Iterate over unique values in column
  for col in one_hot_encoded.columns:
      # Insert one-hot encoded column into original DataFrame
      data.insert(loc=data.columns.get_loc(f'{column}'), column=col , value=one_hot_encoded[col])

  # Drop the original column
  data.drop(f'{column}', axis=1, inplace=True)

In [13]:
import numpy as np
def transformation(df):
    #binning for year to three cateory
    bins = [df['Year'].min(),1990,2010,df['Year'].max()]
    df['Decade'] = pd.cut(df['Year'], bins=bins, labels=['Classic', 'Old', 'New'])
    encoding(df,'Exterior_Color')
    # transformation Sale_End_Date to Date format
    df['Sale_End_Date'] = pd.to_datetime(df['Sale_End_Date'], format='%B %d, %Y %I:%M %p (%Z)',errors='coerce')
    df['Sale_End_Date'].replace(['NaT'], np.nan, inplace=True)
    df.dropna(axis=0,inplace=True)
    return df

In [14]:
df = transformation(df)
di = transformation(di)

In [15]:
print(df.shape)
print(di.shape)

(6880, 45)
(1455, 40)


In [16]:
df.head()

,Vin_no,Make,Model,Year,Mileage,BEIGE,BLACK,BLUE,BROWN,BURGUNDY,...,Drivetrain,Engine,Transmission,Sale_Doc,Keys,Lot_no,Time_Left,Sale_End_Date,Location,Decade
0,**********6114412,Chevrolet,Equinox Lt,2019,99399,0,0,0,0,0,...,AWD,1.5L Turbo DOHC 4-cylinder,AUTOMATIC,CERTIFICATE OF DESTRUCTION(Florida),Master - 1pcs,198147711,19 hours left,2024-05-04 10:00:00-05:00,In Transit,New
1,**********U637747,Hyundai,Tucson Sel,2018,71338,0,0,1,0,0,...,Front-wheel Drive,2.0L 4-Cylinder Petrol,AUTOMATIC,SALVAGE TITLE,YES,197599354,6 days left,2024-05-10 01:30:00-05:00,"Colorado, USA",New
2,**********8128941,Hummer,H3,2008,195198,0,1,0,0,0,...,All wheel drive,3.7L 5-Cylinder Petrol,AUTOMATIC,CERTIFICATE OF TITLE,YES,200038809,19 hours left,2024-05-04 10:00:00-05:00,"New Jersey, USA",Old
3,**********0214513,Mazda,Cx-9,2010,256234,0,0,0,0,0,...,AWD,3.7L DOHC 24-valve V6 engine,AUTOMATIC,SALVAGE(Tennessee),YES,199224369,6 days left,2024-05-10 00:00:00-05:00,"Ohio, USA",Old
4,**********H782018,Land Rover,Range Rover Evoque Pure Plus,2013,149732,0,0,0,0,0,...,AWD,"2L I-4 DI, DOHC, VVT, turbo 240HP",AUTOMATIC,PREVIOUS SALVAGE(Michigan),NO,200159825,6 days left,2024-05-10 01:30:00-05:00,In Transit,New


In [17]:
di.head()

,Vin_no,Make,Model,Year,Mileage,BEIGE,BLACK,BLUE,BROWN,BURGUNDY,...,Drivetrain,Engine,Transmission,Sale_Doc,Keys,Lot_no,Time_Left,Sale_End_Date,Location,Decade
0,**********6114412,Chevrolet,Equinox Lt,2019,99399,0,0,0,0,0,...,AWD,1.5L Turbo DOHC 4-cylinder,AUTOMATIC,SALVAGE(Massachusetts),PRESENT,198147711,19 hours left,2024-05-04 10:00:00-05:00,In Transit,New
1,**********U637747,Hyundai,Tucson Sel,2018,71338,0,0,1,0,0,...,Front-wheel Drive,2.0L 4-Cylinder Petrol,AUTOMATIC,SALVAGE TITLE,YES,197599354,6 days left,2024-05-10 01:30:00-05:00,"Colorado, USA",New
2,**********8128941,Hummer,H3,2008,195198,0,1,0,0,0,...,All wheel drive,3.7L 5-Cylinder Petrol,AUTOMATIC,CERTIFICATE OF TITLE,YES,200038809,19 hours left,2024-05-04 10:00:00-05:00,"New Jersey, USA",Old
3,**********0214513,Mazda,Cx-9,2010,256234,0,0,0,0,0,...,AWD,3.7L DOHC 24-valve V6 engine,AUTOMATIC,SALVAGE(Indiana),PRESENT,199224369,6 days left,2024-05-10 00:00:00-05:00,"Ohio, USA",Old
4,**********H782018,Land Rover,Range Rover Evoque Pure Plus,2013,149732,0,0,0,0,0,...,AWD,"2L I-4 DI, DOHC, VVT, turbo 240HP",AUTOMATIC,CLEAR(Massachusetts),MISSING,200159825,6 days left,2024-05-10 01:30:00-05:00,In Transit,New


# <B>TASK 2.3 **INTEGRATION**

In [18]:
df_concat = pd.concat([df,di],ignore_index=True,verify_integrity=True)
df_concat.shape

(8335, 45)

In [19]:
for col in df_concat.columns:
    if df_concat[col].dtype != 'category':
        df_concat[col].fillna(0,inplace=True)

In [20]:
df_concat.isnull().sum()

Vin_no           0
Make             0
Model            0
Year             0
Mileage          0
BEIGE            0
BLACK            0
BLUE             0
BROWN            0
BURGUNDY         0
BURN             0
CHAMPAGNE        0
CHARCOAL         0
CREAM            0
CRIMSON          0
DARK BLUE        0
DARK BROWN       0
GOLD             0
GRAY             0
GREEN            0
LIGHT BLUE       0
MAROON           0
NAVY             0
ORANGE           0
PEWTER           0
PINK             0
PURPLE           0
RED              0
SILVER           0
TAN              0
TEAL             0
TURQUOISE        0
TWO TONE         0
WHITE            0
YELLOW           0
Drivetrain       0
Engine           0
Transmission     0
Sale_Doc         0
Keys             0
Lot_no           0
Time_Left        0
Sale_End_Date    0
Location         0
Decade           0
dtype: int64

In [21]:
df_concat['RED'].value_counts()

0    7776
1     559
Name: RED, dtype: int64

# 3- Loading

In [23]:
# B. Connect to the MySQL Database:
#  Utilize the create_engine method in Python to establish a connection to the MySQL 
# database installed on your localhost.
# C. Create a Database using Python Code:
#  Write Python code to create a new database within the MySQL server using the 
# established connection.

from sqlalchemy import create_engine,text
import pandas as pd


username = 'root'
password = '12345678'
host = 'localhost'
db = 'CARS'

engine = create_engine(f'mysql://{username}:{password}@{host}/')


with engine.connect() as conn:

    sql_query = f'CREATE DATABASE IF NOT EXISTS {db};'

    try:
        conn.execute(sql_query)
        print(f"Database '{db}' created successfully (if it didn't exist already).")
        engine = create_engine(f'mysql://{username}:{password}@{host}/{db}')
        df_concat.to_sql(name='car', index=False, con=engine)
    except Exception as e:
        print("Error:", e)

Database 'CARS' created successfully (if it didn't exist already).
Error: Table 'car' already exists.


In [25]:

sql_query = text("SELECT Year,Make  FROM car where year >= 2020")
exc = engine.connect()
res = exc.execute(sql_query)
for row in res:
    print(row)

(2021, 'Mazda')
(2022, 'Chevrolet')
(2020, 'Chevrolet')
(2021, 'Bmw')
(2022, 'Chevrolet')
(2023, 'Chevrolet')
(2024, 'Buick')
(2022, 'Buick')
(2022, 'Bmw')
(2023, 'Buick')
(2020, 'Chevrolet')
(2023, 'Chevrolet')
(2024, 'Honda')
(2023, 'Honda')
(2023, 'Mini')
(2021, 'Toyota')
(2021, 'Toyota')
(2020, 'Chevrolet')
(2024, 'Buick')
(2023, 'Chevrolet')
(2020, 'Jeep')
(2021, 'Chevrolet')
(2022, 'Acura')
(2020, 'Acura')
(2022, 'Ford')
(2022, 'Honda')
(2021, 'Bentley')
(2023, 'Genesis')
(2023, 'Acura')
(2021, 'Chevrolet')
(2024, 'Gmc')
(2023, 'Buick')
(2022, 'Ford')
(2020, 'Honda')
(2021, 'Toyota')
(2020, 'Ford')
(2021, 'Mercedes-Benz')
(2020, 'Chevrolet')
(2020, 'Dodge')
(2024, 'Chevrolet')
(2020, 'Kia')
(2021, 'Chevrolet')
(2020, 'Audi')
(2021, 'Audi')
(2020, 'Mercedes-Benz')
(2021, 'Chevrolet')
(2021, 'Gmc')
(2023, 'Audi')
(2021, 'Audi')
(2020, 'Hyundai')
(2021, 'Ford')
(2020, 'Ford')
(2021, 'Honda')
(2022, 'Acura')
(2022, 'Bmw')
(2021, 'Ford')
(2023, 'Hyundai')
(2020, 'Honda')
(2020, 'Bmw')

In [26]:
query = text('select max(Mileage), avg(Mileage), count(Make) from car where Make = "Ford" and Year = 2023')
r = exc.execute(query)
for i in r:
    print(i)

(47440, Decimal('14742.4348'), 23)


In [27]:
query = text('select count(Make) from car where Red = 1')
r = exc.execute(query)
for i in r:
    print(i)

(559,)


In [30]:
# engine = create_engine(f'mysql://{username}:{password}@{host}/{db}')
query = 'select Sale_End_Date from car'
data_df = pd.read_sql_query(query, exc)
data_df

,Sale_End_Date
0,2024-05-04 10:00:00
1,2024-05-10 01:30:00
2,2024-05-04 10:00:00
3,2024-05-10 00:00:00
4,2024-05-10 01:30:00
...,...
8330,2024-05-07 09:30:00
8331,2024-05-07 10:30:00
8332,2024-05-07 10:30:00
8333,2024-05-08 10:24:00
